In [2]:
import ollama
import pandas as pd
import json
import re
import time
import os
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_sentiment_score(comment: str) -> dict:
    """
    返回 dict: {"comment": str, "score": int, "think": str}
    """
    messages = [
        {
            "role": "system",
            "content": (
                "你是一个股市情感分析专家，请对股票评论进行1-5分情感打分（1=极度悲观，2=悲观，3=中性，4=乐观，5=极度乐观）。"
                "你的回答必须是严格的JSON格式，仅包含："
                ' - "think": 简要推理（不超过20字）'
                ' - "score": 整数（1-5）'
                "不要输出任何其他内容、标签或解释。"
            )
        },
        {"role": "user", "content": f"评论：{comment}"}
    ]
    
    try:
        response = ollama.chat(
            model="qwen3:1.7b",
            messages=messages,
            options={"temperature": 0.1}
        )
        raw = response['message']['content'].strip()
        
        # 清理 <think> 标签（兜底）
        cleaned = re.sub(r'(?i)<think>.*?</think>', '', raw, flags=re.DOTALL)
        cleaned = cleaned.strip()
        
        # 提取 JSON
        json_match = re.search(r'\{.*\}', cleaned, re.DOTALL)
        if json_match:
            data = json.loads(json_match.group())
            score = int(data.get("score", 3))
            think = str(data.get("think", ""))[:50]
            if not (1 <= score <= 5):
                score = 3
            return {"comment": comment, "score": score, "think": think}
        else:
            # 回退：尝试提取数字
            match = re.search(r'score["\s]*[:=]\s*(\d)', cleaned, re.IGNORECASE)
            if match:
                return {"comment": comment, "score": int(match.group(1)), "think": "fallback"}
            else:
                return {"comment": comment, "score": 3, "think": "parse_failed"}
                
    except Exception as e:
        return {"comment": comment, "score": 3, "think": f"error:{str(e)[:30]}"}

In [3]:
# === 多线程批量处理 ===
def run_sentiment_analysis(df: pd.DataFrame, max_workers: int = 3) -> pd.DataFrame:
    results = [None] * len(df)
    comments = df['comments'].fillna("").astype(str).tolist()

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # 提交所有任务
        future_to_index = {
            executor.submit(get_sentiment_score, comment): idx
            for idx, comment in enumerate(comments)
        }

        # 收集结果（保持顺序）
        for future in as_completed(future_to_index):
            idx = future_to_index[future]
            try:
                res = future.result()
                results[idx] = res
                print(f"[{idx+1}/{len(df)}] Done → Score: {res['score']}, Think: {res['think']}")
            except Exception as e:
                print(f"Unexpected error at index {idx}: {e}")
                results[idx] = {"comment": comments[idx], "score": 3, "think": "crash"}

    # 提取 score 列
    df = df.copy()
    df['sentiment_score'] = [r['score'] for r in results]
    df['analysis_think'] = [r['think'] for r in results]  # 可选：保存推理
    return df

In [4]:
# ========================
# 批量处理所有 CSV 文件
# ========================
def process_all_csv_files(input_dir: str, output_dir: str, max_workers_per_file: int = 4):
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)

    csv_files = list(input_path.glob("*家电_top3_per_day.csv"))
    if not csv_files:
        print(f"⚠️ 未在 '{input_dir}' 中找到任何 .csv 文件")
        return

    print(f"📂 发现 {len(csv_files)} 个 CSV 文件，开始处理...\n")

    for csv_file in csv_files:
        print(f"📄 正在处理: {csv_file.name}")
        try:
            # 读取 CSV（自动处理编码）
            try:
                df = pd.read_csv(csv_file, encoding='utf-8')
            except UnicodeDecodeError:
                df = pd.read_csv(csv_file, encoding='gbk')

            # 检查是否包含 'comments' 列
            if 'comments' not in df.columns:
                print(f"  ⚠️ 跳过 {csv_file.name}：缺少 'comments' 列")
                continue

            # 执行情感分析
            df_result = run_sentiment_analysis(df, max_workers=max_workers_per_file)

            # 保存结果
            output_file = output_path / f"{csv_file.stem}_sentiment.csv"
            df_result.to_csv(output_file, index=False, encoding='utf-8-sig')
            print(f"  ✅ 已保存: {output_file.name}")

        except Exception as e:
            print(f"  ❌ 处理 {csv_file.name} 时出错: {e}")

    print(f"\n🎉 所有文件处理完成！结果保存至: {output_dir}")

In [5]:
INPUT_DIR = "文本数据合并"
OUTPUT_DIR = "output"

process_all_csv_files(
    input_dir=INPUT_DIR,
    output_dir=OUTPUT_DIR,
    max_workers_per_file=3  # 每个文件内部使用3线程
)

📂 发现 1 个 CSV 文件，开始处理...

📄 正在处理: merged_家电_top3_per_day.csv
[2/552] Done → Score: 3, Think: 评论描述市场行为，未明确表达积极或消极情绪
[1/552] Done → Score: 3, Think: 句子结构混乱，语气疑问，可能涉及策略可行性判断
[3/552] Done → Score: 1, Think: 评论明确表达对垃圾股的不满和长期亏损的绝望
[4/552] Done → Score: 4, Think: 评论中表达了对小米空调性能的质疑，但未达到极端悲观
[6/552] Done → Score: 5, Think: 评论使用积极词汇并表达乐观情绪
[5/552] Done → Score: 4, Think: 评论提及股价频繁涨停，但未明确负面情绪，可能偏中性或乐观。
[7/552] Done → Score: 2, Think: 评论表达对市场走势的悲观预期
[8/552] Done → Score: 1, Think: 评论中使用'硬要'暗示强制亏损，属于极度悲观情绪
[9/552] Done → Score: 1, Think: 评论使用强烈负面词汇表达极度不满
[11/552] Done → Score: 4, Think: 评论认可机构低价减持，暗示市场资金流动，情绪积极
[10/552] Done → Score: 4, Think: 评论强调美的的多角色和行业趋势，整体积极但中性
[12/552] Done → Score: 1, Think: 评论包含强烈负面情绪，反映极度悲观
[13/552] Done → Score: 5, Think: 用户用夸张笑声表达兴奋，可能代表极度乐观
[14/552] Done → Score: 4, Think: 收盘价上涨2分，属于小幅上涨，属于乐观情绪
[16/552] Done → Score: 1, Think: 评论指出长虹美菱三季度空调业绩不及预期，明显负面情绪
[15/552] Done → Score: 3, Think: 评论中'老登'可能带有贬义，但缺乏具体市场信息，难以确定情感倾向。
[17/552] Done → Score: 4, Think: 询问回购是否完成，暗示对回购结果的期待，

In [6]:
# import ollama
# import pandas as pd
# import json
# import re
# import time
# from concurrent.futures import ThreadPoolExecutor, as_completed

# # 示例数据（替换成你的 CSV）
# data = {
#     "author": ["股友15ve310631", "股友15ve310631", "股友15ve310631"],
#     "comments": [
#         "砸毛科技",
#         "这样的板块抄底死了，今天拉不起来",
#         "被套的今天可以跑了，不然阴跌又开始了"
#     ],
#     "reply_comments": ["", "", ""],
#     "reply_authors": ["", "", ""],
#     "update": ["11-27", "11-27", "11-26"]
# }
# df = pd.DataFrame(data)

# # === 执行 ===
# print("🚀 开始多线程情感分析...")
# df_result = run_sentiment_analysis(df, max_workers=4)  # 可调：2~4

# df_result.to_csv("output_with_sentiment.csv", index=False, encoding='utf-8-sig')
# print("\n✅ 分析完成！结果已保存到 output_with_sentiment.csv")